# Chapter 5

# K-Nearest Neighbors Method

## 5.1 A Basic k-NN Classifier

### 5.1.1 The _"Can I eat that?" App_

We start by exploring the mushrooms dataset and looking at a random sample of the data and understand what is there

Dataset URL: https://archive.ics.uci.edu/ml/datasets/mushroom

|  |  |
|--|--|
| Number of Instances | 8124 |
| Number of Attributes |22 |
| Attribute Characteristics | Categorical |
| Missing Values? | Yes |

## Attribute Information:

1. cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
4. bruises?: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d



In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../datasets/mushrooms.csv")
data.sample(10)

,E,F0,F1,F2,F3,F4,F5,F6,F7,F8,...,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21
919,e,x,s,w,f,n,f,w,b,h,...,f,w,w,p,w,o,e,k,s,g
5086,p,f,f,y,f,f,f,c,b,h,...,k,n,b,p,w,o,l,h,y,g
6318,p,f,y,n,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,p
4721,p,x,f,y,f,f,f,c,b,g,...,k,p,p,p,w,o,l,h,v,g
2752,e,x,y,n,t,n,f,c,b,n,...,s,g,p,p,w,o,p,k,y,d
7839,p,k,y,n,f,y,f,c,n,b,...,s,w,p,p,w,o,e,w,v,d
6602,p,x,s,n,f,s,f,c,n,b,...,s,w,w,p,w,o,e,w,v,p
7652,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,n,o,p,b,c,l
3898,p,x,f,w,f,c,f,c,n,p,...,s,w,w,p,w,o,p,n,v,d
967,e,x,f,n,t,n,f,c,b,n,...,s,w,p,p,w,o,p,k,y,d


---
After inspecting the data, we find that we're only intereted in the features labeled as `"F0", "F1", "F2", "F20", "F21"`, so we extract those as our data of interest along with, of course, the label `"E"`

In [2]:
data_of_interest = data.loc[:, ["E", "F0", "F1", "F2", "F20", "F21"]]

data_of_interest = data_of_interest.astype("category")

---
### 5.1.3 How to Measure Similarity?
Before we can measure the Euclidean distance between two mushrooms, we need to map the charcter codes for the features to numeric values in order to plug them into the equation 

In [3]:
def numerically_encode(df):
    
    encoded_df = df.copy()
    encoders = {}
    
    for col in df.columns:
        unique_categories = df.loc[:, col].unique()
        unique_categories.sort() # in-place sorting
    
        encoder = {str: num for num, str in enumerate(unique_categories)}
        encoders[col] = encoder
    
        encoded_df.loc[:, col] = df.loc[:, col].apply(lambda x: encoder[x])
    
    return encoded_df, encoders

In [4]:
def numerically_encode(df):
    return df.apply(lambda x: x.cat.codes)

---
We now encode our data and reterive the encoder dictionary for the label column to see how each label is encoded and be able to interpret the results of the classifier we're going to build.

In [5]:
#encoded_data, encoders = numerically_encode(data_of_interest)
#print(encoders["E"])

encoded_data = numerically_encode(data_of_interest)

encoded_data.sample(5)

,E,F0,F1,F2,F20,F21
4330,1,2,3,9,4,0
3290,0,2,3,3,5,0
452,0,0,2,9,2,3
123,0,2,2,3,0,1
2686,0,5,3,2,5,0


## Exkurs: Visualize clusters

In [6]:
from sklearn.preprocessing import StandardScaler

features = ["F0", "F1", "F2", "F20", "F21"]

x = StandardScaler().fit_transform(encoded_data[features])

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principal_components = pca.fit_transform(x)

principal_df = pd.DataFrame(data = principal_components, columns=["PC1", "PC2"])

principal_df["label"] = data_of_interest["E"]

print("explained variance", pca.explained_variance_ratio_)

principal_df

explained variance [0.25982833 0.22038449]


,PC1,PC2,label
0,1.128953,-0.670754,p
1,0.946066,1.415050,e
2,2.465547,0.834471,e
3,1.965040,-0.392289,p
4,0.975747,0.725656,e
...,...,...,...
8119,1.403368,0.463925,e
8120,-0.348311,-0.398214,e
8121,1.599883,0.464524,e
8122,0.296748,-0.960789,p


In [8]:
import plotly.express as px

fig = px.scatter(principal_df, x="PC1", y="PC2", color="label")

fig

In [16]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, perplexity=50, n_iter=400, learning_rate=100)

X_trans = tsne.fit_transform(x)

tsne_df = pd.DataFrame(data = X_trans, columns=["TSNE1", "TSNE2"])

tsne_df["label"] = data_of_interest["E"]

tsne_df


,TSNE1,TSNE2,label
0,3.377228,-12.638596,p
1,12.993071,-2.321126,e
2,5.226933,-4.782985,e
3,15.676755,12.440513,p
4,-6.608394,18.743679,e
...,...,...,...
8119,-3.750591,13.986289,e
8120,-2.077830,4.503944,e
8121,-3.240177,13.662766,e
8122,-2.607914,26.628428,p


In [17]:
import plotly.express as px

fig = px.scatter(tsne_df, x="TSNE1", y="TSNE2", color="label")

fig

---
### 5.1.4 k-NN in Action
We start our implementation of the k-NN model by implementing a function that calculates the Euclidean distance between two mushrooms 

In [5]:
import math

def d(p1, p2):
    
    N = len(p1)
    squared_distance = 0
    
    for i in range(N):
        squared_distance += (p1[i] - p2[i]) ** 2
        
    return math.sqrt(squared_distance)

In [12]:
def d(p1: np.ndarray, p2: np.ndarray):
    return np.sqrt(sum((x1 - x2) ** 2 for x1, x2 in zip(p1, p2)))


In [17]:
x1, x2 = encoded_data.iloc[:2].values

d(x1, x2)

6.557438524302

---
We can now implement the k-NN classifier by simply calculating the distances between the input we have and all the data in the training set, sort those in ascending fashion by the distance, pick the first k and then report the most common label in our chosen set of k data points.

In [ ]:
from collections import Counter

def knn_classifier_old(new_x, k, X_train, y_train):

    neighbors = [] # a list of tuples (distance_to_new_x, neighbor_label)
    for x, y in zip(X_train, y_train):
        distance = d(x, new_x)
        neighbors.append((distance, y))

    sorted_neighbors = sorted(neighbors, key=lambda n: n[0])
    nearest_k_neighbors = sorted_neighbors[:k]
    
    labels_counter = Counter([label for _, label in nearest_k_neighbors])
    most_voted_label = max(labels_counter.items(), key=lambda i: i[1])
    
    return most_voted_label[0]

In [39]:
from collections import Counter, namedtuple

Neighbor = namedtuple("Neighbor", ["distance", "label"])

def knn_classifier(new_x, k, X_train, y_train):

    neighbors = [] # a list of tuples (distance_to_new_x, neighbor_label)
    for x, y in zip(X_train, y_train):
        distance = d(x, new_x)
        neighbors.append(Neighbor(distance, y))

    nearest_k_neighbors = sorted(neighbors, key=lambda n: n.distance)[:k]
    
    return Counter((n.label for n in nearest_k_neighbors)).most_common(1)[0][0]

---
Before we can test the k-NN classifier, which we did before in chapters 1, 3 with scikit-learn. However, as long as we're doing things from scratch, let's see how we can do it manually

In [21]:
shuffled_data = encoded_data.sample(frac=1., random_state=42)
X, y = shuffled_data.loc[:, 'F0':], shuffled_data.loc[:, "E"]
X, y = X.values, y.values
N = X.shape[0]

X_train, y_train = X[:int(0.75 * N)], y[:int(0.75 * N)]
X_test, y_test = X[int(0.75 * N):], y[int(0.75 * N):]

X_train, y_train, X_test, y_test

(array([[2, 0, 4, 3, 1],
        [2, 2, 2, 4, 2],
        [5, 3, 4, 4, 2],
        ...,
        [5, 2, 3, 0, 1],
        [3, 2, 2, 4, 0],
        [5, 3, 3, 5, 1]], dtype=int8),
 array([0, 1, 1, ..., 0, 1, 1], dtype=int8),
 array([[2, 0, 4, 5, 0],
        [2, 0, 3, 5, 0],
        [5, 2, 0, 4, 1],
        ...,
        [2, 3, 4, 5, 4],
        [3, 2, 2, 4, 4],
        [3, 0, 3, 2, 1]], dtype=int8),
 array([0, 0, 1, ..., 0, 1, 0], dtype=int8))

---
We can now run our classifier and see it in action and how it performs on the test set

In [31]:
# old run
test_preds = [knn_classifier_old(x, 5, X_train, y_train) for x in X_test[:100]]

losses = [1. if y_pred != y else 0. for y_pred, y in zip(test_preds, y_test)]
test_error = (1. / len(test_preds)) * sum(losses)
accuracy = 1. - test_error

print("Test Error {:.3f}, Test Accuracy: {:.3f}".format(test_error, accuracy))

Test Error 0.130, Test Accuracy: 0.870


In [46]:
# new
# old run
test_preds = [knn_classifier(x, 5, X_train, y_train) for x in X_test]

losses = [1. if y_pred != y else 0. for y_pred, y in zip(test_preds, y_test)]
test_error = (1. / len(test_preds)) * sum(losses)
accuracy = 1. - test_error

print("Test Error {:.3f}, Test Accuracy: {:.3f}".format(test_error, accuracy))

Test Error 0.111, Test Accuracy: 0.889


---
While this implementation is giving us a decent performance, it's ineffcient when it comes to time. We can verfiy that by running the `%timeit` magic against the prediction step and see

In [9]:
%timeit [knn_classifier(x, 5, X_train, y_train) for x in X_test]

1min 20s ± 4.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
### 5.1.5 Boosting Performance with NumPy

NumPy fixes the problem with python that causes the above code's ineffciency. Instead of usind dynamically-typed and scattered containers, NumPy relies on data structures that have static pre-defined types which allow them to occupy contiguous memeory locations. These data startuctures are the `ndarray`, which we can find that they are the backbone of our `pandas` data frames.

In [10]:
print(type(X))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


To start using `ndarray`s and utilize their effciency, we first need to understand one of its charactristics that play a key role in determining its behavior, which is its `shape`

In [11]:
print(X.shape)
print(y.shape)

(8124, 5)
(8124,)


---
Shapes determine if we can perform arithmetic operations on `ndarray`s. If the shapes are _compatible_, then arithmetic operation can be performed, and they are performed element-wise like we see in the following example:

In [12]:
import numpy as np

four_ones = np.ones(shape=(4, ))
powers_of_two = np.array([1, 2, 4, 8])

print(four_ones / powers_of_two)

[ 1.     0.5    0.25   0.125]


---
We can take advantage of the broadcasting feature and rewrite our k-NN implementation bye getting rid of the nested for loops for calculating the distance and transfer them from python to NumPy optimized C/C++ code. We also change the voting code to use NumPy alternative and fully harness its powers

In [13]:
def faster_knn_classifier(new_x, k, X_train, y_train):
    
    neighbors_distances = np.sqrt(np.sum((X_train - new_x) ** 2, axis=1))
        
    sorted_neighbors_indecies = np.argsort(neighbors_distances)
    nearest_k_neighbors_indecies = sorted_neighbors_indecies[:k]
    nearest_k_neighbors_labels = y_train[nearest_k_neighbors_indecies]
    
    labels, votes = np.unique(nearest_k_neighbors_labels, return_counts=True)
    most_voted_label_index = np.argmax(votes)
    
    return labels[most_voted_label_index]

---
Now we can test our new k-NN classifier and see how much faster it has become

In [14]:
faster_test_preds = [
    faster_knn_classifier(x, 1, X_train, y_train) for x in X_test
]
faster_losses = (faster_test_preds != y_test)
faster_test_error = np.mean(faster_losses)

print("Test Error {:.4f}, Test Accuracy: {:.4f}".format(
    faster_test_error, 
    1. - faster_test_error
))

Test Error 0.1176, Test Accuracy: 0.8824


In [15]:
%timeit [faster_knn_classifier(x, 5, X_train, y_train) for x in X_test]

1.41 s ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
## 5.2 A Better k-NN Classifier
### 5.2.2 Using k-d Tress with scikit-learn
After we saw how k-d trees can boost the nearst neighbors search, let's see that in action by using scikit-learn's implementation of the k-NN model that comes with a k-d trees implementation

In [21]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=5, algorithm='kd_tree')
classifier.fit(X_train, y_train)

sklearn_test_accuracy = classifier.score(X_test, y_test)

print("Test Error {:.4f}, Test Accuracy: {:.4f}".format(
    1. - sklearn_test_accuracy, 
    sklearn_test_accuracy
))

Test Error 0.1176, Test Accuracy: 0.8824


In [22]:
%timeit classifier.score(X_test, y_test)

21.1 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 5.2.3 Tuning the Value of k

To start searching through the possible values of k in order to find the value that would return the best performing model, we need to craete another held out dataset that is used during this serach, thus leaving our test unseen. 

In [23]:
new_X_train, X_valid = X_train[:5525], X_train[5525:]
new_y_train, y_valid = y_train[:5525], y_train[5525:]

---
With this new **_validation set_**, we can start looking through the best k value by iterating through the possible values and training a k-NN classifier and recording the best performing one on the training set

In [24]:
best_score, best_k, best_classifier = 0., None, None
for k in range(1, 21):
        classifier = KNeighborsClassifier(n_neighbors=k)
        classifier.fit(new_X_train, new_y_train)
        score = classifier.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_k = k
            best_classifier = classifier

print("Best k: {}, Best Validation Score: {:.4f}".format(best_k, best_score))
print("Test Accuracy: {:.4f}".format(best_classifier.score(X_test, y_test)))

Best k: 11, Best Validation Score: 0.8950
Test Accuracy: 0.9005


---
We can extend our linear search tuning algorithm into a grid search algorithm that searches through pairs of hyperparameters and reports the best configuration. The second hyperparameter we're going to search for is the `metric` and we're gonna see that automatic choice aligns with our expert's judgment

In [25]:
best_score, best_k, best_metric, best_classifier = 0., -1, None, None
for k in range(1, 21):
    for metric in ['euclidean', 'manhattan', 'hamming', 'canberra']:
        classifier = KNeighborsClassifier(n_neighbors=k, metric=metric)
        classifier.fit(new_X_train, new_y_train)
        score = classifier.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_k = k
            best_metric = metric
            best_classifier = classifier

print("Best k: {}, Best Metric: {}, Best Validation Score: {:.4f}".format(
    best_k, 
    best_metric, 
    best_score
))
print("Test Accuracy: {:.4f}".format(best_classifier.score(X_test, y_test)))

Best k: 11, Best Metric: hamming, Best Validation Score: 0.9017
Test Accuracy: 0.9065
